In [48]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re


In [49]:
df_wide = pd.read_csv('../data/dengue.csv')
df_wide.head()

,period,101 TAPLEJUNG,102 SANKHUWASABHA,103 SOLUKHUMBU,104 OKHALDHUNGA,105 KHOTANG,106 BHOJPUR,107 DHANKUTA,108 TERHATHUM,109 PANCHTHAR,...,610 SURKHET,701 BAJURA,702 BAJHANG,703 DARCHULA,704 BAITADI,705 DADELDHURA,706 DOTI,707 ACHHAM,708 KAILALI,709 KANCHANPUR
0,W24 2019Sun 2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,W25 2019Sun 2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,W26 2019Sun 2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,W27 2019Sun 2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,W28 2019Sun 2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
# We keep 'period' and transform the 77 district columns into 'District' and 'Cases'
district_columns = df_wide.columns.drop('period').tolist()
df_long = pd.melt(
    df_wide, 
    id_vars=['period'], 
    value_vars=district_columns, 
    var_name='District', 
    value_name='Cases'
)

In [51]:
df_long.head()

,period,District,Cases
0,W24 2019Sun 2019,101 TAPLEJUNG,NaN
1,W25 2019Sun 2019,101 TAPLEJUNG,NaN
2,W26 2019Sun 2019,101 TAPLEJUNG,NaN
3,W27 2019Sun 2019,101 TAPLEJUNG,NaN
4,W28 2019Sun 2019,101 TAPLEJUNG,NaN


In [52]:
df_long.shape
# df_long.to_csv('../data/dengue_long.csv', index=False)

(25179, 3)

In [53]:
def extract_time_features(text):
    """
    Extracts Year and Week from strings like 'W40 2025Sun 2025' or 'W1 2020Tue 2020'.
    """
    # Convert to string to handle any potential non-string inputs
    text = str(text)
    
    # Regex breakdown:
    # W(\d+)   -> Matches 'W' followed by one or more digits (Captures Week)
    # \s+      -> Matches one or more whitespace characters
    # (\d{4})  -> Matches exactly four digits (Captures the first occurrence of Year)
    match = re.search(r'W(\d+)\s+(\d{4})', text)
    
    if match:
        week_num = int(match.group(1))
        year_val = int(match.group(2))
        return pd.Series([year_val, week_num])
    else:
        # Returns None if the pattern isn't found to avoid breaking the script
        return pd.Series([None, None])

# Apply the function to the entire 'period' column
# This creates two new columns: 'Year' and 'Week'
df_long[['Year', 'Week']] = df_long['period'].apply(extract_time_features)

In [54]:
# Verification step
print("Sample of extracted time features:")
print(df_long[['period', 'Year', 'Week']].head())

Sample of extracted time features:
             period  Year  Week
0  W24 2019Sun 2019  2019    24
1  W25 2019Sun 2019  2019    25
2  W26 2019Sun 2019  2019    26
3  W27 2019Sun 2019  2019    27
4  W28 2019Sun 2019  2019    28


In [55]:
# remove the period column as it's no longer needed, sintead add a count column that counts cases for each district, move the year and week in left of district
# reorder columns
df_long = df_long[['Year', 'Week', 'District', 'Cases']]
df_long.head() 

,Year,Week,District,Cases
0,2019,24,101 TAPLEJUNG,NaN
1,2019,25,101 TAPLEJUNG,NaN
2,2019,26,101 TAPLEJUNG,NaN
3,2019,27,101 TAPLEJUNG,NaN
4,2019,28,101 TAPLEJUNG,NaN


In [56]:
df_long.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25179 entries, 0 to 25178
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Year      25179 non-null  int64  
 1   Week      25179 non-null  int64  
 2   District  25179 non-null  object 
 3   Cases     6285 non-null   float64
dtypes: float64(1), int64(2), object(1)
memory usage: 787.0+ KB


In [57]:
# count the number of cases in each district by summing the values in the 'Cases' column for each district
cases_by_district = df_long.groupby('District')['Cases'].sum().reset_index()
cases_by_district

,District,Cases
0,101 TAPLEJUNG,53.0
1,102 SANKHUWASABHA,1819.0
2,103 SOLUKHUMBU,26.0
3,104 OKHALDHUNGA,403.0
4,105 KHOTANG,422.0
...,...,...
72,705 DADELDHURA,498.0
73,706 DOTI,244.0
74,707 ACHHAM,555.0
75,708 KAILALI,1128.0
